In [4]:
# Generate 18 PNG images (no HTML) with matplotlib 3D surface
# Colors like the example: blue->white->red (coolwarm) with symmetric colorbar
# Axes: x=time (t), y=moneyness (S0/K), z=MR
# View: roughly like the example (adjust elev/azim if you want)

import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# --- files follow your naming ---
models = ["GBM", "Heston", "Jump"]
risks  = ["ele", "elw", "ente", "entl", "entw", "esl"]

# --- titles you requested ---
title_map = {
    "ele":  "EL with Exp",
    "elw":  "EL with Weibull type",
    "entl": "ERM with Linear",
    "ente": "ERM with Exp",
    "entw": "ERM with Weibull type",
    "esl":  "ES with Linear",
}

out_dir = "plots_surfaces_png"
os.makedirs(out_dir, exist_ok=True)

# camera/view similar to your screenshot
ELEV = 28
AZIM = -60

for model in models:
    for risk in risks:
        csv_name = f"surface_{model}_{risk}.csv"
        if not os.path.exists(csv_name):
            print(f"[SKIP] missing: {csv_name}")
            continue

        df = pd.read_csv(csv_name)

        # numeric safety
        df["t"] = pd.to_numeric(df["t"], errors="coerce")
        df["moneyness"] = pd.to_numeric(df["moneyness"], errors="coerce")
        df["MR"] = pd.to_numeric(df["MR"], errors="coerce")
        df = df.dropna(subset=["t", "moneyness", "MR"])

        # pivot to grid: rows=moneyness, cols=time
        pivot = df.pivot_table(index="moneyness", columns="t", values="MR", aggfunc="mean")

        m = pivot.index.to_numpy(dtype=float)
        t = pivot.columns.to_numpy(dtype=float)
        Z = pivot.to_numpy(dtype=float)

        # sort axes
        m_sort = np.argsort(m)
        t_sort = np.argsort(t)
        m = m[m_sort]
        t = t[t_sort]
        Z = Z[m_sort][:, t_sort]

        # meshgrid for plot_surface
        T, M = np.meshgrid(t, m)

        # symmetric diverging color scale like the example
        vmax = np.nanmax(np.abs(Z))
        if not np.isfinite(vmax) or vmax == 0:
            vmax = 1.0
        vmin = -vmax

        fig = plt.figure(figsize=(7.5, 5.5), dpi=200)
        ax = fig.add_subplot(111, projection="3d")

        surf = ax.plot_surface(
            T, M, Z,
            cmap="coolwarm",
            vmin=vmin, vmax=vmax,
            linewidth=0,
            antialiased=True
        )

        ax.set_title(f"{title_map[risk]} ({model})")
        ax.set_xlabel("time (t)")
        ax.set_ylabel("moneyness (S0/K)")
        ax.set_zlabel("maturity risk (MR)")

        ax.view_init(elev=ELEV, azim=AZIM)

        # similar framing; feel free to tweak
        ax.set_box_aspect((1.15, 1.0, 0.7))

        cbar = fig.colorbar(surf, ax=ax, shrink=0.65, pad=0.08)
        cbar.set_label("MR")

        out_png = os.path.join(out_dir, f"surface_{model}_{risk}.png")
        plt.tight_layout()
        plt.savefig(out_png, bbox_inches="tight")
        plt.close(fig)

        print(f"[OK] {out_png}")

print("Done.")


[OK] plots_surfaces_png\surface_GBM_ele.png
[OK] plots_surfaces_png\surface_GBM_elw.png
[OK] plots_surfaces_png\surface_GBM_ente.png
[OK] plots_surfaces_png\surface_GBM_entl.png
[OK] plots_surfaces_png\surface_GBM_entw.png
[OK] plots_surfaces_png\surface_GBM_esl.png
[OK] plots_surfaces_png\surface_Heston_ele.png
[OK] plots_surfaces_png\surface_Heston_elw.png
[OK] plots_surfaces_png\surface_Heston_ente.png
[OK] plots_surfaces_png\surface_Heston_entl.png
[OK] plots_surfaces_png\surface_Heston_entw.png
[OK] plots_surfaces_png\surface_Heston_esl.png
[OK] plots_surfaces_png\surface_Jump_ele.png
[OK] plots_surfaces_png\surface_Jump_elw.png
[OK] plots_surfaces_png\surface_Jump_ente.png
[OK] plots_surfaces_png\surface_Jump_entl.png
[OK] plots_surfaces_png\surface_Jump_entw.png
[OK] plots_surfaces_png\surface_Jump_esl.png
Done.


In [8]:
# ============================================================
# HEATMAPS (process sensitivity): DeltaMR on (param1,param2)
# 3 figures total (GBM/Heston/Jump), each with 12 panels:
#   top 6  -> t=0.2 (2x3)
#   bottom 6 -> t=0.8 (2x3)
# Horizontal separator line.
#
# Improvements requested:
# 1) Better spacing between panels + cleaner layout
# 2) Better colorbars (one per risk) placed OUTSIDE and aligned
# 3) Cells with +/-inf are NOT white:
#    - They are clipped to +/-vmax (so they become the max color)
#    - NaNs (if any) are shown as dark and also pushed to max color (same intent)
# 4) Robust per-risk scaling (percentile) shared across both t
# ============================================================

import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D

# -------- config --------
models = ["GBM", "Heston", "Jump"]
t_list = [0.2, 0.8]
risks  = ["ele", "elw", "ente", "entl", "entw", "esl"]

title_map = {
    "ele":  "EL with Exp",
    "elw":  "EL with Weibull type",
    "entl": "ERM with Linear",
    "ente": "ERM with Exp",
    "entw": "ERM with Weibull type",
    "esl":  "ES with Linear",
}

param_axes = {
    "GBM":   ("mu", "sigma"),
    "Heston":("kappa", "theta"),
    "Jump":  ("meanJ", "lam"),
}

# robust scaling percentile (reduces saturation from outliers)
PCTL = 95  # try 90 if still too saturated

out_dir = "plots_heatmaps_png_risk_scaled_v2"
os.makedirs(out_dir, exist_ok=True)

for model in models:
    xcol, ycol = param_axes[model]

    # ---- load all dfs for this model ----
    dfs = {}  # dfs[(t, risk)] = df
    risk_absvals = {r: [] for r in risks}

    for t_idx in t_list:
        for risk in risks:
            fname = f"sens_process_{model}_{risk}_t{t_idx}.csv"
            if not os.path.exists(fname):
                print(f"[SKIP] missing: {fname}")
                continue

            df = pd.read_csv(fname)

            df[xcol] = pd.to_numeric(df[xcol], errors="coerce")
            df[ycol] = pd.to_numeric(df[ycol], errors="coerce")
            df["DeltaMR"] = pd.to_numeric(df["DeltaMR"], errors="coerce")

            # keep inf for later clipping; drop rows missing x/y
            df = df.dropna(subset=[xcol, ycol, "DeltaMR"])

            dfs[(t_idx, risk)] = df

            vals = df["DeltaMR"].to_numpy()
            # use only finite values to compute robust scale
            vals_f = vals[np.isfinite(vals)]
            if vals_f.size:
                risk_absvals[risk].append(np.abs(vals_f))

    if not dfs:
        print(f"[SKIP] no data for model={model}")
        continue

    # ---- per-risk scale (shared across both t) ----
    risk_scale = {}
    for risk in risks:
        if len(risk_absvals[risk]) == 0:
            risk_scale[risk] = (-1.0, 1.0)
            continue

        abs_vals = np.concatenate(risk_absvals[risk])
        abs_vals = abs_vals[np.isfinite(abs_vals)]
        if abs_vals.size == 0:
            risk_scale[risk] = (-1.0, 1.0)
            continue

        vmax = np.percentile(abs_vals, PCTL)
        if not np.isfinite(vmax) or vmax == 0:
            vmax = float(np.nanmax(abs_vals)) if np.nanmax(abs_vals) > 0 else 1.0

        risk_scale[risk] = (-vmax, vmax)

    # ---- create figure with extra right margin for colorbars ----
    fig = plt.figure(figsize=(15, 14), dpi=220)
    gs = fig.add_gridspec(
        nrows=4, ncols=3,
        left=0.06, right=0.84, top=0.92, bottom=0.07,  # leave space on right for colorbars
        wspace=0.22, hspace=0.30                         # spacing between panels
    )

    axes = np.empty((4, 3), dtype=object)
    for r in range(4):
        for c in range(3):
            axes[r, c] = fig.add_subplot(gs[r, c])

    fig.suptitle(
        f"{model} | DeltaMR heatmaps (risk-scaled) | top: t=0.2, bottom: t=0.8",
        fontsize=14
    )

    cmap = plt.get_cmap("coolwarm")

    # keep a mappable per risk for the colorbar (shared mapping)
    risk_mappable = {}

    for ti, t_idx in enumerate(t_list):
        row0 = 0 if ti == 0 else 2

        for i, risk in enumerate(risks):
            r = row0 + (i // 3)
            c = (i % 3)
            ax = axes[r, c]

            vmin, vmax = risk_scale[risk]
            ax.set_title(title_map[risk], fontsize=11)

            key = (t_idx, risk)
            if key not in dfs:
                ax.axis("off")
                ax.text(0.5, 0.5, f"missing\n{t_idx}", ha="center", va="center")
                continue

            df = dfs[key]
            piv = df.pivot_table(index=ycol, columns=xcol, values="DeltaMR", aggfunc="mean")

            x_vals = np.array(piv.columns, dtype=float)
            y_vals = np.array(piv.index, dtype=float)
            Z = piv.to_numpy(dtype=float)

            # sort
            xs = np.argsort(x_vals)
            ys = np.argsort(y_vals)
            x_vals = x_vals[xs]
            y_vals = y_vals[ys]
            Z = Z[ys][:, xs]

            # ---- handle inf / nan:
            # replace +inf/-inf/nan by sign*vmax (or +vmax for nan) then clip
            Z_clean = np.array(Z, copy=True)

            pos_inf = np.isposinf(Z_clean)
            neg_inf = np.isneginf(Z_clean)
            nan_mask = np.isnan(Z_clean)

            Z_clean[pos_inf] = vmax
            Z_clean[neg_inf] = vmin
            Z_clean[nan_mask] = vmax  # your intent: "darken / push to max color"

            Z_clean = np.clip(Z_clean, vmin, vmax)

            extent = [x_vals.min(), x_vals.max(), y_vals.min(), y_vals.max()]
            im = ax.imshow(
                Z_clean,
                origin="lower",
                aspect="auto",
                extent=extent,
                cmap=cmap,
                vmin=vmin,
                vmax=vmax,
                interpolation="nearest",
            )

            ax.set_xlabel(xcol)
            ax.set_ylabel(ycol)

            ax.set_xticks(x_vals)
            ax.set_yticks(y_vals)
            ax.tick_params(axis="x", rotation=45)

            # store one representative mappable per risk for colorbar (only once)
            if risk not in risk_mappable:
                risk_mappable[risk] = im

    # ---- horizontal separator line between halves ----
    top_block_bottom = axes[1, 0].get_position().y0
    bottom_block_top = axes[2, 0].get_position().y1
    y_line = (top_block_bottom + bottom_block_top) / 2.0
    fig.add_artist(Line2D([0.06, 0.84], [y_line, y_line], transform=fig.transFigure, linewidth=2))

    # ---- add 6 aligned slim colorbars on the right (clean spacing) ----
    # stacked vertically, each labeled by risk
    cb_left = 0.87
    cb_width = 0.018
    cb_top = 0.90
    cb_bottom = 0.10
    cb_gap = 0.012
    total_h = cb_top - cb_bottom
    each_h = (total_h - (len(risks) - 1) * cb_gap) / len(risks)

    for j, risk in enumerate(risks):
        if risk not in risk_mappable:
            continue

        bottom = cb_top - (j + 1) * each_h - j * cb_gap
        cax = fig.add_axes([cb_left, bottom, cb_width, each_h])
        cb = fig.colorbar(risk_mappable[risk], cax=cax)
        cb.set_label(risk, rotation=90)
        cb.ax.tick_params(labelsize=8)

    out_png = os.path.join(out_dir, f"heatmap_{model}_t02_t08_risk_scaled_v2.png")
    plt.savefig(out_png, bbox_inches="tight")
    plt.close(fig)

    print(f"[OK] {out_png}")

print("Done.")



[OK] plots_heatmaps_png_risk_scaled_v2\heatmap_GBM_t02_t08_risk_scaled_v2.png
[OK] plots_heatmaps_png_risk_scaled_v2\heatmap_Heston_t02_t08_risk_scaled_v2.png
[OK] plots_heatmaps_png_risk_scaled_v2\heatmap_Jump_t02_t08_risk_scaled_v2.png
Done.


In [10]:
# ============================================================
# LINE PLOTS (aversion sensitivity): DeltaMR vs param
# 3 figures total (GBM/Heston/Jump), each with 12 panels:
#   top  6 -> t=0.2
#   bottom 6 -> t=0.8
# Horizontal separator line.
#
# CHANGE REQUEST:
#   - Use ente -> a (instead of gamma)
#   - Use entw -> k (instead of gamma)
#
# Reads CSVs from BLOCO 4:
#   sens_aversion_{MODEL}_{RISK}_{PARAM}_t{t}.csv
# Saves PNGs.
# ============================================================

import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D

models = ["GBM", "Heston", "Jump"]
t_list = [0.2, 0.8]
risks  = ["ele", "elw", "ente", "entl", "entw", "esl"]

title_map = {
    "ele":  "EL with Exp",
    "elw":  "EL with Weibull type",
    "entl": "ERM with Linear",
    "ente": "ERM with Exp",
    "entw": "ERM with Weibull type",
    "esl":  "ES with Linear",
}

# param used per risk in BLOCO 4 filenames (UPDATED)
risk_param = {
    "entl": "gamma",
    "ente": "a",     # <-- changed
    "entw": "k",     # <-- changed
    "ele":  "a",
    "elw":  "k",
    "esl":  "beta",
}

# robust y scaling per risk (across both t) using percentile on finite values
PCTL = 95

out_dir = "plots_lines_aversion_png"
os.makedirs(out_dir, exist_ok=True)

for model in models:
    # ---- load all dfs for this model ----
    dfs = {}  # dfs[(t, risk)] = df
    risk_absvals = {r: [] for r in risks}

    for t_idx in t_list:
        for risk in risks:
            param = risk_param[risk]
            fname = f"sens_aversion_{model}_{risk}_{param}_t{t_idx}.csv"
            if not os.path.exists(fname):
                print(f"[SKIP] missing: {fname}")
                continue

            df = pd.read_csv(fname)

            df["param_value"] = pd.to_numeric(df["param_value"], errors="coerce")
            df["DeltaMR"] = pd.to_numeric(df["DeltaMR"], errors="coerce")
            df = df.dropna(subset=["param_value", "DeltaMR"])
            df = df.sort_values("param_value")

            dfs[(t_idx, risk)] = df

            vals = df["DeltaMR"].to_numpy()
            vals_f = vals[np.isfinite(vals)]
            if vals_f.size:
                risk_absvals[risk].append(np.abs(vals_f))

    if not dfs:
        print(f"[SKIP] no data for model={model}")
        continue

    # ---- y-scale per risk (shared across both t) ----
    risk_scale = {}
    for risk in risks:
        if len(risk_absvals[risk]) == 0:
            risk_scale[risk] = (-1.0, 1.0)
            continue

        abs_vals = np.concatenate(risk_absvals[risk])
        abs_vals = abs_vals[np.isfinite(abs_vals)]
        if abs_vals.size == 0:
            risk_scale[risk] = (-1.0, 1.0)
            continue

        vmax = np.percentile(abs_vals, PCTL)
        if not np.isfinite(vmax) or vmax == 0:
            vmax = float(np.nanmax(abs_vals)) if np.nanmax(abs_vals) > 0 else 1.0
        risk_scale[risk] = (-vmax, vmax)

    # ---- figure layout ----
    fig = plt.figure(figsize=(15, 14), dpi=220)
    gs = fig.add_gridspec(
        nrows=4, ncols=3,
        left=0.07, right=0.97, top=0.92, bottom=0.07,
        wspace=0.28, hspace=0.38
    )

    axes = np.empty((4, 3), dtype=object)
    for r in range(4):
        for c in range(3):
            axes[r, c] = fig.add_subplot(gs[r, c])

    fig.suptitle(f"{model} | Aversion sensitivity: DeltaMR vs parameter (top: t=0.2, bottom: t=0.8)", fontsize=14)

    for ti, t_idx in enumerate(t_list):
        row0 = 0 if ti == 0 else 2

        for i, risk in enumerate(risks):
            r = row0 + (i // 3)
            c = (i % 3)
            ax = axes[r, c]

            param = risk_param[risk]
            ax.set_title(f"{title_map[risk]}  |  param: {param}", fontsize=11)

            key = (t_idx, risk)
            if key not in dfs:
                ax.axis("off")
                ax.text(0.5, 0.5, f"missing\n{t_idx}", ha="center", va="center")
                continue

            df = dfs[key]
            x = df["param_value"].to_numpy(dtype=float)
            y = df["DeltaMR"].to_numpy(dtype=float)

            vmin, vmax = risk_scale[risk]

            # push inf/nan to extremes (so it shows as max effect)
            y_clean = np.array(y, copy=True)
            y_clean[np.isposinf(y_clean)] = vmax
            y_clean[np.isneginf(y_clean)] = vmin
            y_clean[np.isnan(y_clean)] = vmax
            y_clean = np.clip(y_clean, vmin, vmax)

            ax.plot(x, y_clean, linewidth=2)
            ax.axhline(0.0, linewidth=1)

            ax.set_xlabel(param)
            ax.set_ylabel("DeltaMR")
            ax.set_ylim(vmin, vmax)

            if param == "beta":
                ax.set_xticks(x)

    # ---- horizontal separator between halves ----
    top_block_bottom = axes[1, 0].get_position().y0
    bottom_block_top = axes[2, 0].get_position().y1
    y_line = (top_block_bottom + bottom_block_top) / 2.0
    fig.add_artist(Line2D([0.07, 0.97], [y_line, y_line], transform=fig.transFigure, linewidth=2))

    out_png = os.path.join(out_dir, f"lines_aversion_{model}_t02_t08.png")
    plt.savefig(out_png, bbox_inches="tight")
    plt.close(fig)

    print(f"[OK] {out_png}")

print("Done.")


[OK] plots_lines_aversion_png\lines_aversion_GBM_t02_t08.png
[OK] plots_lines_aversion_png\lines_aversion_Heston_t02_t08.png
[OK] plots_lines_aversion_png\lines_aversion_Jump_t02_t08.png
Done.
